# Введение в регрессионный анализ

## Практикум 6. Линейные модели с эффектами взаимодействия, выгрузка результатов регрессионного анализа

*Алла Тамбовцева*

### Подготовка к работе: импорт необходимых библиотек

Импортируем библиотеку `pandas` для загрузки и обработки данных и функцию `ols()` из `statsmodels` для построения линейных моделей.

In [2]:
import pandas as pd
from statsmodels.formula.api import ols

### Сюжет 1: модель с эффектом взаимодействия

Давайте построим модель с эффектом взаимодействия из статьи **How Do Politicians Bargain? Evidence from Ultimatum Games with Legislators in Five Countries**, которая обсуждалась на лекции. В статье анализируются результаты игры «Ультиматум», которая состоит в следующем (по крайней мере, ее основная часть сводится к этому):

* Политикам из разных стран (Бельгия, Канада, Чехия, Германия, США) дают 1000$, какую-то часть этой суммы они могут предложить другому участнику.
* Другой участник может принять предлагаемые деньги или часть суммы. 
* При этом другой участник может быть политиком или избирателем, однопартийцем предлагающего деньги или представителем другой партии.

Интересно посмотреть, насколько отличаются стратегии поведения политиков по отношению к политикам и избирателям, однопартийцам и представителям других партий, то есть, другими словами, какой категории (политики-однопартийцы, избиратели-однопартийцы, политики из других партий/избиратели из других партий) политики готовы отдавать большие суммы. 

Загрузим данные с результатами игры из CSV-файла:

In [3]:
ug = pd.read_csv("ug_replication_data.csv")
ug.head()

,Unnamed: 0,V3,ug_propose,ug_accept,ug_p2_copartisan,ug_p2_copartisan_labelled,ug_p2_politician,ug_p2_politician_labelled,ug_propose_first,pol,country,country_name,sex,age_cat,gov_exp,tenure
0,1,VL1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,BE,Belgium,1.0,41-50,NaN,12.0
1,2,VL1002,500.0,500.0,1.0,Co-Partisan,1.0,Politician,0.0,1,BE,Belgium,0.0,21-40,NaN,5.0
2,3,VL1010,500.0,450.0,1.0,Co-Partisan,0.0,Non-Politician,0.0,1,BE,Belgium,1.0,61-70,NaN,24.0
3,4,VL1012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,BE,Belgium,0.0,41-50,NaN,12.0
4,5,VL1013,500.0,300.0,1.0,Co-Partisan,0.0,Non-Politician,0.0,1,BE,Belgium,1.0,51-60,NaN,5.0


Ключевые переменные:
    
* `ug_propose`: сумма, которую участник готов предложить;
* `ug_accept`: сумма, которую другой участник готов принять;
* `ug_p2_copartisan`: предлагает ли участник деньги однопартийцу или нет (1 или 0);
* `ug_p2_politician`: предлагает ли участник деньги политику или избирателю (1 или 0);
* `ug_propose_first`: предлагает ли участник деньги первым;
* `pol`: участник-политик или нет (здесь все 1, отобраны все политики);
* `country`: страна, из которой участник игры.

В коде для репликации результатов статьи есть этап отбора наблюдений, чтобы гарантировать, что мы отобрали только участников-политиков, повторим этот этап (хотя, как можно убедиться, в файле у нас уже только политики):

In [4]:
# лишняя осторожность не помешает

ug_filt = ug[ug["pol"] == 1]

Прежде чем переходить к модели с эффектом взаимодействия, давайте посмотрим на более простую модель, которая описывает, как величина предлагаемой суммы зависит от того, предлагает ли участник деньги политику или избирателю, однопартийцу или представителю другой партии. Учтем также, что на предлагаемую сумму также может влиять принадлежность к определенной стране (политики из разных стран, в среднем, предлагают разные суммы) и то, предлагает ли политик деньги первым или нет:

In [5]:
mod00 = ols("ug_propose ~ ug_propose_first + country + ug_p2_politician + ug_p2_copartisan", 
            ug_filt).fit()

In [6]:
print(mod00.summary())

                            OLS Regression Results                            
Dep. Variable:             ug_propose   R-squared:                       0.077
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     9.700
Date:                Mon, 16 Oct 2023   Prob (F-statistic):           1.23e-11
Time:                        20:08:25   Log-Likelihood:                -5745.2
No. Observations:                 827   AIC:                         1.151e+04
Df Residuals:                     819   BIC:                         1.154e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          548.2171     22.373  

Проинтерпретируем полученные результаты.

* `Intercept`: 548 – это среднее ожидаемое значение зависимой переменной в случае, когда все независимые равны 0. Что это содержательно означает? В среднем, 548 долларов другим участникам готовы предлагать политики из Бельгии (базовая категория, в выдаче пропущена, `CA`, `CH`, `DE` и `US` равны 0), которые не предлагают деньги первыми (`ug_propose_first` равно 0), которые при этом предлагают деньги избирателям (`ug_p2_politician` равно 0) из других партий (`ug_p2_copartisan` равно 0).

* Коэффициенты при `country[T.CA]`–`country[T.US]` показывают различия в средних значениях зависимой переменной между выбранной категорией и базовой. То есть насколько отличаются, при прочих равных условиях, в среднем, суммы, предлагаемые политиками из Канады/Чехии/Германии/США по сравнению с политиками из Бельгии. На 5%-ном уровне значимости отличным от 0 можно считать только оценку коэффициента при Чехии (p-value меньше 0.05, 95%-ный доверительный интервал включает 0), политики из Чехии, в среднем, предлагают на 64-65 долларов меньше, чем политики из Бельгии. 

* Коэффициент при `ug_p2_politician`: оценка значима на любом из конвенциональных уровней значимости, это разница в средних значениях зависимой переменной в случае, когда `ug_p2_politician` равно 0 и 1. То есть, при прочих равных условиях, в среднем, другим политикам политики готовы предлагать на 110-111 долларов меньше, чем избирателям.

* Коэффициент при `ug_p2_copartisan`: оценка значима на любом из конвенциональных уровней значимости, это разница в средних значениях зависимой переменной в случае, когда `ug_p2_copartisan` равно 0 и 1. То есть, при прочих равных условиях, в среднем, однопартийцам политики готовы предлагать на 84-85 долларов больше.

Важное замечание: $R^2$ у модели довольно низкий, модель объясняет примерно 8% дисперсии зависимой переменной. Но это не означает, что модель плохая. Независимые переменные в модели все бинарные, их изменчивость сама по себе небольшая (всего два значения 0 и 1), поэтому их включение и не должно сильно увеличивать коэффициент детерминации. К тому же, наша задача здесь – не построить идеальную модель с максимально возможной прогностической силой, наша задача – посмотреть на коэффициенты при определенных переменных, чтобы сделать выводы о стратегиях участников, поэтому важен не $R^2$ как формальный показатель, а сама спецификация модели, ее сравнение с другими аналогичными моделями.


Теперь добавим в модель эффект взаимодействия переменных `ug_p2_copartisan` и `ug_p2_politician` в предположении, что на величину предлагаемой суммы влияет не столько принадлежность к партии и принадлежность к группе избирателей по отдельности, сколько их совместный эффект. Другими словами, от двух независимых сравнений *избиратель-политик* и *однопартиец-представитель другой партии* мы переходим к более интересному сравнению четырех групп: избиратели-однопартийцы, избиратели из других партий, политики-однопартийцы, политики из других партий. 

В Python в формуле внутри `ols()` эффект взаимодействия можно добавить по-разному. Первый способ – указать переменные, чье взаимодействие мы учитываем, через `:`:

In [7]:
# +\ – добавила слэш для разбиения уравнения по строкам
# в одну строчку длинно и некрасиво

mod01 = ols("ug_propose ~ ug_propose_first + country + ug_p2_politician + \
            ug_p2_copartisan + ug_p2_politician : ug_p2_copartisan", 
            ug_filt).fit()
print(mod01.summary())

                            OLS Regression Results                            
Dep. Variable:             ug_propose   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     8.881
Date:                Mon, 16 Oct 2023   Prob (F-statistic):           1.03e-11
Time:                        20:09:38   Log-Likelihood:                -5743.7
No. Observations:                 827   AIC:                         1.151e+04
Df Residuals:                     818   BIC:                         1.155e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc

При первом способе важно не потерять переменные, которые входят во взаимодействие, и включить их в модель по отдельности (обратите внимание, сами переменные `ug_p2_politician` и `ug_p2_copartisan` в уравнении присутствуют). Иначе модель с эффектом взаимодействия теряет смысл. 

Второй способ – указать переменные взаимодействия через `*`:

In [8]:
mod01 = ols("ug_propose ~ ug_propose_first + country + ug_p2_politician * ug_p2_copartisan", 
    ug_filt).fit()
print(mod01.summary())

                            OLS Regression Results                            
Dep. Variable:             ug_propose   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     8.881
Date:                Mon, 16 Oct 2023   Prob (F-statistic):           1.03e-11
Time:                        20:09:43   Log-Likelihood:                -5743.7
No. Observations:                 827   AIC:                         1.151e+04
Df Residuals:                     818   BIC:                         1.155e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc

При таком способе определения модели Python сам добавит `ug_p2_politician` и `ug_p2_copartisan` по-отдельности. Идея такая: `y ~ x + z + x:z` и `y ~ x * z` – это эквивалентные способы задать уравнение модели с эффектом взаимодействия `x` и `z`, которые на выходе дадут модель вида `y ~ x + z + x * z`.

Перейдем к интерпретации.

Оценку коэффициента при взаимодействии `ug_p2_politician:ug_p2_copartisan` можно считать значимой на 10%-ном уровне значимости. Получается, действительно, эти переменные оказывают совместный эффект на зависимую переменную. 

Давайте запишем в сокращенном виде уравнение модели (нас интересуют коэффициенты только при последних трех переменных в выдаче) и сравним стратегии политиков.

$$
\widehat{\text{ug_propose}_i} = 530 + \dots - 79 \times \text{politician}_i + 117 \times \text{copartisan}_i - 61 \times \text{politician}_i \times \text{copartisan}_i
$$

Рассмотрим четыре случая.

**Случай 1.**  `politician = 0` и `copartisan = 0`, политик предлагает деньги избирателю из другой партии; политик предлагает какую-то сумму, если это политик из Бельгии, который не предлагает деньги первым (`ug_propose_first` равен 0), то, в среднем, от него можно ожидать 530 долларов.

**Случай 2.** `politician = 0` и `copartisan = 1` политик предлагает деньги избирателю из той же партии; политик, в среднем, предлагает сумму на 117 долларов выше, чем в предыдущем случае, когда получатель – избиратель из другой партии (подставьте 0 и 1 в уравнение).

**Случай 3.** `politician = 1` и `copartisan = 0` политик предлагает деньги политику из другой партии; политик, в среднем, предлагает сумму на 79 долларов ниже, чем в случае, когда получатель – избиратель из другой партии (подставьте 1 и 0 в уравнение).

**Случай 4.** `politician = 1` и `copartisan = 1` политик предлагает деньги политику из своей партии; политик, в среднем, предлагает сумму на 23 доллара ниже, чем в случае, когда получатель – избиратель из другой партии (подставьте 1 и 1 в уравнение, получите `-79 + 117 - 61`).

В целом, получается, что политики склонны предлагать большие суммы избирателям (даже если они из другой партии), чем политикам (даже если они из той же партии).

Вычислим предельные эффекты `politician` и `copartisan` по отдельности.

Предельный эффект `politician` на `ug_propose`:

$$
\text{marginal effect} = - 79 - 61 \times \text{copartisan}_i.
$$

Если будем подставлять вместо `copartisan` 0 или 1, мы получим, что величина предельного эффекта `politician` на `ug_propose` равна $-$79 в случае, если деньги предлагаются представителю другой партии, и $-79-61 = -140$ в случае, если деньги предлагаются однопартийцу. 

Что такое предельный эффект `politician` на `ug_propose`? При прочих равных условиях, это разница в средних значениях `ug_propose` в случаях, если `politician = 0` и `politician = 1`. Итого, получаем, что разница в средних суммах, которые участники готовы отдавать политикам и избирателям, зависит от принадлежности последних к партии. Если деньги предлагаются представителям своей партии, эта разница меньше на 61 доллар.

Предельный эффект `copartisan` на `ug_propose`:

$$
\text{marginal effect} = 117 - 61 \times \text{politician}_i.
$$

Если будем подставлять вместо `politician` 0 или 1, мы получим, что величина предельного эффекта `copartisan` на `ug_propose` равна 117 в случае, если деньги предлагаются избирателю, и $117-61 = 56$ в случае, если деньги предлагаются политику. 

Что такое предельный эффект `copartisan` на `ug_propose`? При прочих равных условиях, это разница в средних значениях `ug_propose` в случаях, если `copartisan = 0` и `copartisan = 1`. Итого, получаем, что разница в средних суммах, которые участники готовы отдавать однопартийцам и не-однопартийцам, зависит от того, являются ли последние политиками или избирателями. Если деньги предлагаются политикам, эта разница больше на 56 долларов.

### Сюжет 2: выгрузка результатов в HTML и LaTeX

Установим библиотеку `stargazer` для экспорта результатов регрессионного анализа в красивом табличном виде (от *star* – «звезда» и *gaze* – «глазеть», так как в выдачах присутствуют звездочки для отметки значимости):

In [ ]:
!pip install stargazer

Импортируем класс `Stargazer`:

In [9]:
from stargazer.stargazer import Stargazer

С помощью этого класса мы можем подготовить выдачу в табличном виде, поместив необходимые модели внутрь `Stargazer()` в виде списка:

In [10]:
stargazer = Stargazer([mod00, mod01])

Посмотрим, что получилось:

In [11]:
stargazer

Красивая выдача, каждая модель в отдельном столбце, как обычно и бывает в статьях. Как ее выгрузить? Если вы работаете в LaTeX, все просто, метод `.render_latex()` вернет код LaTeX для таблицы, который можно сводобно скопировать:

In [12]:
print(stargazer.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{2}{c}{\textit{Dependent variable: ug_propose}} \
\cr \cline{2-3}
\\[-1.8ex] & (1) & (2) \\
\hline \\[-1.8ex]
 Intercept & 548.217$^{***}$ & 530.660$^{***}$ \\
& (22.373) & (24.551) \\
 country[T.CA] & 14.288$^{}$ & 15.079$^{}$ \\
& (33.193) & (33.156) \\
 country[T.CH] & -64.731$^{**}$ & -64.323$^{**}$ \\
& (30.756) & (30.720) \\
 country[T.DE] & -1.117$^{}$ & 4.071$^{}$ \\
& (33.787) & (33.880) \\
 country[T.US] & 3.742$^{}$ & 3.173$^{}$ \\
& (21.417) & (21.393) \\
 ug_p2_copartisan & 84.976$^{***}$ & 117.374$^{***}$ \\
& (17.613) & (25.718) \\
 ug_p2_politician & -110.605$^{***}$ & -79.746$^{***}$ \\
& (17.697) & (25.134) \\
 ug_p2_politician:ug_p2_copartisan & & -61.164$^{*}$ \\
& & (35.419) \\
 ug_propose_first & 0.741$^{}$ & 2.008$^{}$ \\
& (17.709) & (17.703) \\
\hline \\[-1.8ex]
 Observations & 827 & 827 \\
 $R^2$ & 0.077 & 0.080 \\
 Adjusted $R^2$ & 0.069 &

Если вы работаете в Word, можно вернуть код для таблицы в виде кода HTML:

In [13]:
print(stargazer.render_html())

<table style="text-align:center"><tr><td colspan="3" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="2"><em>Dependent variable: ug_propose</em></td></tr><tr><td style="text-align:left"></td><tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td></tr>
<tr><td colspan="3" style="border-bottom: 1px solid black"></td></tr>

<tr><td style="text-align:left">Intercept</td><td>548.217<sup>***</sup></td><td>530.660<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(22.373)</td><td>(24.551)</td></tr>
<tr><td style="text-align:left">country[T.CA]</td><td>14.288<sup></sup></td><td>15.079<sup></sup></td></tr>
<tr><td style="text-align:left"></td><td>(33.193)</td><td>(33.156)</td></tr>
<tr><td style="text-align:left">country[T.CH]</td><td>-64.731<sup>**</sup></td><td>-64.323<sup>**</sup></td></tr>
<tr><td style="text-align:left"></td><td>(30.756)</td><td>(30.720)</td></tr>
<tr><td style="text-align:left">country[T.DE]</td><td>-

Далее, в блокноте или прямо в Jupyter (*New* – *Text File*) можно создать новый текстовый файл с расширением `.htm` и скопировать туда код для таблицы. Это специфический формат, и как страницу HTML в браузере можно открыть, и как текст в редакторе в обработанном виде. Если открыть этот файл в браузере, мы увидим готовую сверстанную таблицу, если открыть с помощью Word или аналогичного текстового редактора, мы увидим таблицу, которую можно редактировать. 

При необходимости (а часто такая необходимость возникает, хотя бы для того, чтобы вывести значения с точностью до второго знака после точки) можно скорректировать настройки выгружаемых таблиц. Почитать про это можно в [тьюториале](https://github.com/StatsReporting/stargazer/blob/master/examples.ipynb) из официальной [документации](https://pypi.org/project/stargazer/).